# Fun with Psi4

*Psi4* is a quantum mechanics package that does the same job as *Gamess*. It is open source (*Gamess* is freely available but is proprietary) and can communicate with *Python* so we can use it directly via *Jupyter* notebooks. 

This exercise assumes that you are up and running with Psi4. See the guide to isntallation and startup to get here. First let us import the *Psi4* API for *Python* and some other essential tools using the code below.

As we progress you may find the need for more information. I suggest consulting the **[*Psi4* website](https://psicode.org)** and the documentation and Q&A forums within. 

In [2]:
# use psi4conda environment
import psi4
import os
import numpy as np
print('done')

done


## First Things First

We need to set up some important parameters first before we run any calculations. These can be changed at any time as we proceed.

In the code below we will be calling functions from the ```psi4``` module to set the memory limit allowed to the program (some calculations might take over all your RAM unless you set a limit), set the number of threads (the number of processor cores you want to make available) and declare the fineame for the output file.

In the ```psi4.set_output_file()``` function we see the following parameters used: 
 - The first is a string that gives the file name and the second is a flag that sets the mode to append to the file with each call in subsequent functions or to overwrite. It is best to set ```append=True```. These two parameters are required.
 - The remaining parameters are optional. Setting ```loglevel=20``` will result in events ate the "info" level of logging priority be written to the log file (the log file will have the same name as the output file and will have the file extension ".log".) The other parameters are described in the **[documentation](https://psicode.org/psi4manual/master/external_apis.html#psi4.set_output_file)**.


In [3]:
psi4.set_memory("1GB")
psi4.set_output_file("p4_output4.out", append=False, loglevel=20, print_header=False, inherit_loglevel=True, execute=True)
psi4.core.set_num_threads(4)


  Memory set to 953.674 MiB by Python driver.


## Your First Molecule

### Building Using a Z-Matrix

To do anything we need atoms in an arrangement that can be interpreted by the various tools within *Psi4*. The software uses cartesian coordinates but the data can be entered in a number of formats. Below is an example Z-matrix that describes a starting point for water. Note that we guestimated the bond lengths and bond angle.

The ```psi4.geometry()``` function will take the input (a text string) and convert it into a ```psi4.core.Molecule``` object. This object will include the original data and also contain the calculated cartesian coordinates, aligned with the symmetry axis that was declared or was detected. This data object has many methods that allow it to be manipulated. A few will be demonstrated below. See the **[documentation](https://psicode.org/psi4manual/master/api/psi4.core.Molecule.html)** for more info.

In the code below I will demonstrate two different sets of initial data for a water molecule. First I will use a Z-matrix and second I will use a cartesian xyz matrix. In both cases I will write out the xyz data generated by the ```psi4.geometry()``` as it created the ```Molecule``` data object.

Run the command below and then open the corresponding ```.out``` file. Several commands below print output to that file. ```molecule.print_out()``` will punch the xyz cartessian coordinated generated by the ```psi4.geometry()``` function. The other commands do as they advertize. You can use similar print methods to record information in the output file at various stages.

In [32]:
# Declare output file so we start fresh with empty file
psi4.set_output_file("H2O-Z-matrix.out", append=False, loglevel=20, print_header=False, inherit_loglevel=True, execute=True)

# The Z-matrix as a text string
data = """
       0 1
       O1
       H2  1  length
       H3  1  length  2  angle
       length = 1.0
       angle = 105
       units angstrom
       """ 

# Create the Molecule object
molecule = psi4.geometry(data)             # Create Molecule object from data string

# These print methods in a Molecule object print to the output file
molecule.print_out()
molecule.print_distances()
molecule.print_bond_angles()
molecule.print_in_input_format()

# These print commands print to the console
print(molecule.point_group().full_name())
print(molecule.point_group().symbol())

energy = psi4.energy("hf/cc-pvdz")
print("The energy for this configuration is {:.7f} Hartree".format(energy))

C2v(Z)
c2v
The energy for this configuration is -76.0213449 Hartree


### Your First Output File

We now have an output file that results from the commands above. There are many commands that will punch information into the output file. The series of ```molecule.print...``` commands demonstrated above are just a few examples.

Examine the output file (```H2O-Z-matrix.out```) and you will see that the atoms were placed in the *yz* plane with the *z*-axis being the primary axis of symmetry and the *xz*-plane being the plane of reflection along the primary axis. We can report the point groups determined by *Psi4* using the ```molecule.point_group()``` commands at the end of the block above retrun the symmetry point group. ```full_name()``` returns the point group with the primary axis noted and ```symbol()``` returns just the point group (These designations are listed in the output file as a retult of the ```molecule.print_out()``` command.

Output files will be very large once we start using more powerful commands that calculate energy, optimize structures and report wavefunctions and vibrational data.

### Changing the Molecule

We can change variables in the molecule object if they were used in the input string. Above there is the variable ```angle``` that was set to 105 degrees. Let us change it to 107 degrees and get the new energy. Consider the code below.

In [34]:
if molecule.is_variable("angle"):  # A test to see if the variable is present


    before = molecule.get_variable("angle")
    print(f"Before change: {before:.3f} degrees")   # Current angle
    molecule.set_variable("angle", 120)             # Change angle
    after = molecule.get_variable("angle")          
    print(f"After change: {after:.3f} degrees")     # Current angle
    
    molecule.update_geometry()     # Apply the change
    molecule.symmetrize(1E-4)      # Analyze symmetry after the change
    print(molecule.point_group().symbol())

energy2 = psi4.energy("hf/cc-pvdz")
print("The energy for this configuration is {:.7f} Hartree".format(energy))

e_diff = (energy2 - energy) * psi4.constants.hartree2kJmol
print(f"The difference in energy between in initial and chnaged geomtery is {e_diff:.2f} kJ/mole\n")

molecule.print_out()             # Print changed geometry to outpur file
molecule.print_distances()
molecule.print_bond_angles()
molecule.print_in_input_format()



Before change: 120.000 degrees
After change: 120.000 degrees
c2v
The energy for this configuration is -76.0213449 Hartree
The difference in energy between in initial and chnaged geomtery is 18.63 kJ/mole



### Building Using xyz Coordinates

We can repeat the same build of water and use Cartesian coordinates instead. Usually we would build the molecule in a graphical interface like *Avogadro* and export the structure as a set of *xyz* coordinates. Avogadro will produce an input file for a *Psi4* calculation. We can cut and paste the coordinates into our python program.

In simple cases like water, we can just throw out some initial coordinates based on what we know. We can start with the centra oxygen at the origin of the coordinate system (0,0,0). I can then place the hydrogens on the *xy* plane (or any of the three planes) by setting +*x*,+*y* and +*x*,–*y* 

In [5]:
# Declare output file so we start fresh with empty file
psi4.set_output_file("H2O-Cart.out", append=False, loglevel=20, print_header=False, inherit_loglevel=True, execute=True)

# The coordinates are a text string
data = """
       0 1
       O1  0     0     0
       H2  x     y     0
       H3  x    -y     0
       x = 0.7
       y = 0.7
       units angstrom
       """ 

# Create the Molecule object
molecule = psi4.geometry(data)             # Create Molecule object from data string

# These print methods in a Molecule object print to the output file
molecule.print_out()
molecule.print_distances()
molecule.print_bond_angles()
molecule.print_in_input_format()

# These print commands print to the console
print(molecule.point_group().full_name())
print(molecule.point_group().symbol())


C2v(Z)
c2v


### Examine the Output File

The output file for the commands above is ```H2O-Cart.out```. Examine this file and you will see that the ```psi4.geometry()  ``` command took the coordinates and translated then to position the center of mass at the origin (note the non-zero value for oxygen on the *z*-axis) and rotated the atoms in the coordinate system to align the principal axis of symmetry with the z-axis and to have the mirror plane along that axis to be the *xz* plane. Again *Psi4* determined the symmetry point group and primary axis.

## Conclusion

We have created our first inputs for *Psi4* using a Z-matrix and an array of xyz coordinates. We have used commands built into the ```Molecule``` object to punch information itno the output file. We are now ready to move into the next phase, obtaining a structure where the geometry has been optimized and determining its energy.

In [ ]:
print(molecule.is_variable("angle"))

print(type(molecule))
molecule.update_geometry()
molecule.print_out()
molecule.symmetrize(1E-4)
molecule.update_geometry()

print(molecule.get_variable("angle"))
molecule.set_variable("angle", 104)
print(molecule.get_variable("angle"))

a = molecule.symmetry_from_input()
print(a)
#molecule.set_point_group("C2V")

In [15]:
energy = psi4.energy("hf/cc-pvdz")
print("The energy for this configuration is {:.7f} Hartree".format(energy))

The energy for this configuration is -76.0255897 Hartree
